In [1]:
import notebooks_setup

notebooks_setup.setup()

Added /Users/arjunbawa/Desktop/cluster-links to Python path.
Added /Users/arjunbawa/Desktop/cluster-links/src to Python path
Current working directory: /Users/arjunbawa/Desktop/cluster-links/notebooks


In [2]:
from data_collection.models import load_bookmarks_from_json

bookmarks = []
for bookmark_json_file in notebooks_setup.DATA_OUTPUT_DIR.glob("bookmarks-*.json"):
    bookmarks.append(load_bookmarks_from_json(bookmark_json_file))

print([type(i) for i in bookmarks])

[<class 'data_collection.models.BookmarksRoot'>, <class 'data_collection.models.BookmarksRoot'>]


In [3]:
print(type(bookmarks[0].children), type(bookmarks[0].children[0]))
print(type(bookmarks[1].children), type(bookmarks[1].children[0]))
print(
    list(map(lambda x: len(x.children), bookmarks)),
    list(map(lambda x: len(x.links), bookmarks)),
    list(map(lambda x: x.title, bookmarks)),
)

<class 'list'> <class 'data_collection.models.BookmarkFolder'>
<class 'list'> <class 'data_collection.models.BookmarkFolder'>
[37, 1] [0, 3] ['Unnamed', 'Unnamed']


In [4]:
from copy import deepcopy

bm_root = deepcopy(bookmarks[0])

for i in range(1, len(bookmarks)):
    assert bm_root.title == bookmarks[i].title
    bm_root.links.extend(bookmarks[i].links)
    bm_root.children.extend(bookmarks[i].children)

print(
    list(map(lambda x: len(x.children), bookmarks)),
    list(map(lambda x: len(x.links), bookmarks)),
)
print(len(bm_root.children), len(bm_root.links))

assert len(bm_root.children) == sum(map(lambda x: len(x.children), bookmarks))

assert len(bm_root.links) == sum(map(lambda x: len(x.links), bookmarks))

bookmarks = bm_root

[37, 1] [0, 3]
38 3


In [5]:
import pandas as pd

df = (
    pd.DataFrame(
        map(lambda x: (x["domain"], 1), bookmarks.get_flat_bookmarks()),
        columns=["domain", "count"],
    )
    .groupby("domain")
    .count()
    .reset_index()
    .replace(r"^\s*$", None, regex=True)
    .dropna()
    .reset_index(drop=True)
)

df = df.join(
    pd.DataFrame(df["domain"].str.split(".").map(lambda x: x[::-1]).to_list()).rename(
        columns=lambda x: f"domain_{x}"
    )
)

df_most_freq = df.loc[df["count"] >= df["count"].quantile(0.99)].sort_values(
    by="count", ascending=False
)[["domain", "count"]]

df_most_freq.to_csv(
    notebooks_setup.DATA_OUTPUT_DIR.joinpath("significant_domains.csv"), index=False
)
df_most_freq

,domain,count
1659,www.youtube.com,703
431,github.com,381
1281,www.google.com,356
1495,www.reddit.com,322
957,towardsdatascience.com,291
670,myanimelist.net,195
991,uwaterloo.ca,118
354,en.wikipedia.org,84
336,ece.uwaterloo.ca,64
232,cs.uwaterloo.ca,42


In [6]:
import re
from IPython.display import display

for c in list(filter(lambda c: re.search(r"domain_\d+", c), df.columns)):
    _df = df[[c, "count"]].groupby(c).sum().sort_values(by="count", ascending=False)
    display(_df.loc[_df["count"] >= _df["count"].quantile(0.95)])

,count
domain_0,
com,3721
ca,620
org,422
edu,280
net,270


,count
domain_1,
youtube,712
github,457
google,447
uwaterloo,346
reddit,329
...,...
co,8
arduino,8
rsc,8


,count
domain_2,
www,2476
cs,120
en,86
ece,74
math,71
m,48
docs,42
ocw,29
learn,21


,count
domain_3,
www,149
en,39
student,32
web,7
webdocs,6


,count
domain_4,
www,43


In [7]:
import json

with open(
    notebooks_setup.DATA_OUTPUT_DIR.joinpath("combined-flat-list.json"), "w"
) as f:
    f.write(json.dumps(bookmarks.get_flat_bookmarks(), indent=2))